In [1]:
import os

data_path = '../data/collected_data/09_06_2023'

for filename in os.listdir(data_path):
	f = os.path.join(data_path, filename)
	if os.path.isfile(f):
		print(f)


../data/collected_data/09_06_2023/UE00_60_min_09062023.xlsx
../data/collected_data/09_06_2023/UE00_0_min_09062023.xlsx
../data/collected_data/09_06_2023/UE00_30_min_09062023.xlsx
../data/collected_data/09_06_2023/UE00_15_min_09062023.xlsx
../data/collected_data/09_06_2023/E00_09062023.xlsx
../data/collected_data/09_06_2023/UE00_45_min_09062023.xlsx
